In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings
sys.path.append("../src/") # go to parent dir
from main import get_X, get_y, get_score, get_train_test
# from models import get_model_benchmark1, get_model_benchmark2, show_importance
from models.model import Model
from models.benchmark import Benchmark1, Benchmark2
warnings.filterwarnings('ignore')

## Read Train data

In [ ]:
train_data = get_X('train')
train_scores = get_y()
test_data = get_X('test')

## Benchmark

We try to model if AWAY wins

### train, valid test split

In [ ]:
X_train, y_train, X_test, y_test, X_valid, y_valid, target = get_train_test(train_size=0.8, random_state=42)

### xgb model

In [ ]:
%load_ext autoreload
%autoreload 2
benchmark1 = Benchmark1(X_train, y_train, X_valid, y_valid, train_scores)
benchmark1.train()
benchmark1.evaluate(X_test)
# benchmark1.save(test_data)

In [ ]:
%load_ext autoreload
%autoreload 2
benchmark2 = Benchmark2(X_train, y_train, X_valid, y_valid, train_scores)
benchmark2.train()
benchmark2.evaluate(X_test)

In [ ]:
benchmark2.save(test_data)

## Submission

In [ ]:
test_data = get_X('test')

In [ ]:
X_bench = xgb.DMatrix(test_data)

predictions = bst.predict(X_bench, iteration_range=(0, bst.best_iteration))
predictions = pd.DataFrame(predictions)

predictions[2] = 0
predictions.columns = [0,2,1]
predictions = (predictions.reindex(columns=[0,1,2]).rank(1,ascending=False)==1).astype(int)

predictions.columns = ['HOME_WINS', 'DRAW', 'AWAY_WINS']
predictions.index = test_data.index
submission = predictions.reset_index()
submission.to_csv('./benchmark_submission.csv', index=False)
submission.head()